#Monster API LLM Integration


In [ ]:
!pip install -e ../../ --quiet
!pip install pydantic==1.10.9 --quiet
!python3 -m pip install monsterapi --quiet
!python3 -m pip install sentence_transformers

In [17]:
import os
#os.environ["MONSTER_API_KEY"] = "{}"
from llama_index.llms import MonsterLLM
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.embeddings import LangchainEmbedding
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

/home/vikas/Documents/qblocks/llama_index/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
llm = MonsterLLM(model = "falcon-7b-instruct", temperature = 0.75)
result = llm.complete("Who are you?")
print(result)

INFO:monsterapi.client:Calling Monster API with url: https://api.monsterapi.ai/v1/generate/falcon-7b-instruct, with payload: {'prompt': 'Who are you?', 'top_k': 40, 'top_p': 0.9, 'temp': 0.98, 'max_length': 256, 'repetition_penalty': 1.2, 'beam_size': 1}


 I'm a sophisticated AI system designed to be your assistant, I exist to serve you the best I can.


##RAG Approach

In [19]:
embed_model = LangchainEmbedding(HuggingFaceEmbeddings())
service_context = ServiceContext.from_defaults(
    chunk_size=1024, llm=llm, embed_model=embed_model
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
